# Run **Meshed Tree Protocol** on FABRIC Nodes

## Input Required Information

| Variable | Use |
| --- | --- |
| SLICE_NAME    | Name of the slice you wish to work on. |
| NODE_TO_FAIL | Node which will lose access to an interface. |
| NETWORK_TO_FAL | The network holding the node to fail, which will determine the interface to fail. To get the network name, locate the slice_log.json for your slice and then pick out the network based on your experiement needs.

In [ ]:
SLICE_NAME = "mtp_update"
NODE_TO_FAIL = "L2"
NETWORK_TO_FAIL = "core-L2-S1"

## Access the Slice

The orchestrator class is initalized, which also means the slice and its nodes are now accessable as well.

In [ ]:
from FabUtils import FabOrchestrator

try:
    manager = FabOrchestrator(SLICE_NAME)
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
import json
 
# Opening JSON file
f = open(f'{SLICE_NAME}_slice_log.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)

print("Configuration data loaded.")

## Delete the Log from a Prior Test if Necessary

In [ ]:
rmLogCmd = "rm MTP_*.log"
manager.executeCommandsParallel(rmLogCmd, prefixList="L,S")

## MTP **Initial Convergence**

Delay by a bit to get everything working first. The spines are started first, then 5 seconds later the leaves are.

In [ ]:
import time

startHelloTime = round(time.time()*1000) + 3 * 60 * 1000

startCmd = f"tmux new-session -d -s mtp 'cd ~/SRC; sudo ./MTPstart -t {startHelloTime}'"

# First on Spines
manager.executeCommandsParallel(startCmd, prefixList="S")
print("Started MTP on spines")

# Wait a bit
time.sleep(5)

# Then on Leaves
manager.executeCommandsParallel(startCmd, prefixList="L")
print("Started MTP on Leaves")

# MTP **Reconvergence** Testing

## Take the Interface down

In [ ]:
# Take the specified interface down
intfName = data[NODE_TO_FAIL]["networks"][NETWORK_TO_FAIL]["intf"]
failingNode = manager.slice.get_node(NODE_TO_FAIL)
failingNode.execute(f"sudo ip link set {intfName} down")

print(f"{NODE_TO_FAIL} had interface {intfName} taken down.")

## Stop MTP on Each Node

In [ ]:
stopCmd = "tmux kill-session -t mtp"
manager.executeCommandsParallel(stopCmd, prefixList="L,S")

## Collect Log Results

Now that the nodes have logged updates to their respective log files, they need to be downloaded to be analyzed.

In [ ]:
import os

LOG_DIR_PATH = "../logs"
LOG_NAME = "MTP_{name}.log"
logPath = os.path.join(LOG_DIR_PATH, LOG_NAME)

# If the logs directory does not already exist, create it
if not os.path.exists(LOG_DIR_PATH):
    os.makedirs(LOG_DIR_PATH)
    
manager.downloadFilesParallel(logPath, LOG_NAME, prefixList="L,S", addNodeName=True)

## Bring the Interface Back Up

In [ ]:
failingNode.execute(f"sudo ip link set {intfName} up")
print(f"{NODE_TO_FAIL} had interface {intfName} brought back up.")